## ex10 - Utilisation des vues pour simplifier les requêtes

L'un des beaux aspects du modèle de données relationnel et SQL est que la sortie d'une requête est également une table, une relation pour être précis. 

Il peut s'agir d'une seule colonne ou d'une seule ligne, mais il s'agit néanmoins d'un tableau. 

Une vue est une requête qui peut être utilisée comme une table. Une vue peut être considérée comme une table virtuelle qui ne contient pas de données. 

Elle correspond juste à une requête. Chaque fois qu'une vue est accédée, la requête sous-jacente est exécutée et les résultats renvoyés peuvent être utilisés comme s'ils constituaient une table réelle.

Il y a plusieurs raisons (http://www.sqlitetutorial.net/sqlite-create-view/) pour utiliser les vues. Gardez à l’esprit le principe de programmation DRY: ne vous répétez pas. Éviter les répétitions permet de gagner du temps et d'éviter les erreurs inutiles. C'est l'une des bonnes raisons pour lesquelles nous enregistrons les requêtes sous forme de vues de base de données réutilisables.

Les vues SQLite sont créées à l'aide de l'instruction CREATE VIEW. 

Les vues SQLite peuvent être créées à partir d'une seule table, de plusieurs tables ou d'une autre vue. 

Voici la syntaxe de la commande ***CREATE VIEW*** de base (http://www.sqlitetutorial.net/sqlite-create-view/):

>CREATE [TEMP | TEMPORARY] VIEW view_name AS<br>
>SELECT column1, column2.....<br>
>FROM table_name<br>
>WHERE [condition];


La vue dans SQLite est en lecture seule. Cela signifie que vous ne pouvez pas utiliser les instructions INSERT, DELETE et UPDATE pour mettre à jour les données dans les tables de base via la vue.


In [ ]:
%load_ext sql

In [ ]:
from google.colab import drive
# drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)


Mounted at /content/gdrive


### 1. Connection à la database demo.db3

In [ ]:
%sql sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3

'Connected: @/content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3'

Si vous ne vous souvenez pas des tables présentes dans la database de démonstration, vous pouvez toujours utiliser la commande suivante pour les retrouver.

In [ ]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///data/demo.db3
Done.


name
rch
hru
sub
sed
watershed_daily
watershed_monthly
watershed_yearly
channel_dimension
hru_info
sub_info


### 2. Simplifying queries with views

In the previous notebook, we used CASE and Subquery to calculate seasonal runoff from the table of rch. Here we use a view to simplify the calculation.

#### 2.1 Have a recall of how to calculate season runoffs 

In [ ]:
%%sql sqlite://
SELECT RCH, Quarter, AVG(FLOW_OUTcms) as Runoff
FROM(
SELECT RCH, YR, 
CASE 
    WHEN (MO) BETWEEN 3 AND 5 THEN 'MAM'   
    WHEN (MO) BETWEEN 6 and 8 THEN 'JJA'
    WHEN (MO) BETWEEN 9 and 11 THEN 'SON'
    ELSE 'DJF' 
END Quarter,
FLOW_OUTcms
from rch)
GROUP BY RCH, Quarter
LIMIT 5

Done.


RCH,Quarter,Runoff
1,DJF,99.2049905353
1,JJA,1405.26229799
1,MAM,559.746932019
1,SON,454.737985314
2,DJF,56.3285390854


#### 2.2 Creating a view

In [ ]:
%%sql sqlite://
CREATE VIEW RCH_VW AS SELECT RCH, YR, 
CASE 
    WHEN (MO) BETWEEN 3 AND 5 THEN 'MAM'   
    WHEN (MO) BETWEEN 6 and 8 THEN 'JJA'
    WHEN (MO) BETWEEN 9 and 11 THEN 'SON'
    ELSE 'DJF' 
END Quarter,
FLOW_OUTcms
from rch

Done.


[]

Let's query the SSN_RCH view

In [ ]:
%%sql sqlite://
SELECT * 
FROM RCH_VW 
LIMIT 5

Done.


RCH,YR,Quarter,FLOW_OUTcms
1,1981,DJF,146.252487183
2,1981,DJF,96.1828536987
3,1981,DJF,11.8613681793
4,1981,DJF,49.4065132141
5,1981,DJF,272.106018066


### 2.3 Recalculate seasonal runoffs with views

:) The codes really really get shorter.

In [ ]:
%%sql sqlite://
SELECT RCH, Quarter, AVG(FLOW_OUTcms) as Runoff
FROM RCH_VW
GROUP BY RCH, Quarter
LIMIT 5

Done.


RCH,Quarter,Runoff
1,DJF,99.2049905353
1,JJA,1405.26229799
1,MAM,559.746932019
1,SON,454.737985314
2,DJF,56.3285390854


### 2.4 Deleting Views

It is quite easy to delete views. Just drop it like the following.

In [ ]:
%sql DROP VIEW RCH_VW

 * sqlite:///data/demo.db3
Done.


[]

### Summary

Views are virtual tables that do not hold data, only SQL statements. Those statements are executed each time the view is accessed. Because views are created dynamically as they are accessed and the data in those views are always fresh and up-to-date, they have some advantages over creating a subtables from a table. The data in subtables is static and could be out-to-date.

A view is useful in some [cases](http://www.sqlitetutorial.net/sqlite-create-view/):
- First, views provide an abstraction layer over tables. You can add and remove the columns in the view without touching the schema of the underlying tables.
- Second, you can use views to encapsulate complex queries with joins to simplify the data access.